In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/MyDrive/'

In [ ]:
df_train=pd.read_csv(path + 'train.csv')
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def process_text(texts):
    """
    Process a list of text strings by tokenizing, converting to lower case, lemmatizing,
    and removing stopwords and non-alphabet characters.

    Args:
    texts (list of str): List of text strings to be processed.

    Returns:
    list of str: List of processed text strings.
    """
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    processed_texts = []

    for text in texts:
        # Tokenize and remove non-alphabetical characters
        tokens = word_tokenize(text)
        tokens = [re.sub('[^A-Za-z]', '', token) for token in tokens]
        tokens = [token for token in tokens if token]  # Remove empty strings

        # Convert to lower case and lemmatize
        tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words]

        # Join tokens back into a single string
        processed_text = ' '.join(tokens)
        processed_texts.append(processed_text)

    return processed_texts

In [ ]:
df_train['processed_text_q1'] = process_text(df_train['question1'].astype(str))
df_train['processed_text_q2'] = process_text(df_train['question2'].astype(str))
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate,processed_text_q1,processed_text_q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math math divided
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water


In [ ]:
MAX_NB_WORDS = 300000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df_train['processed_text_q1'].values.astype(str))+list(df_train['processed_text_q2'].values.astype(str)))

In [ ]:
from sklearn.model_selection import train_test_split
X=df_train.drop(columns=['is_duplicate'],axis=1)
y=df_train['is_duplicate']
X_trained,X_tested,y_trained,y_tested=train_test_split(X,y,test_size=0.2,random_state=420)

In [ ]:
# X_train_q1 = tokenizer.texts_to_sequences(np.array(listq1))
X_train_q1 = tokenizer.texts_to_sequences(X_trained['processed_text_q1'].values.astype(str))
X_train_q1 = pad_sequences(X_train_q1, maxlen = 30, padding='post')

# X_train_q2 = tokenizer.texts_to_sequences(np.array(listq2))
X_train_q2 = tokenizer.texts_to_sequences(X_trained['processed_text_q2'].values.astype(str))
X_train_q2 = pad_sequences(X_train_q2, maxlen = 30, padding='post')

In [ ]:
# X_test_q1 = tokenizer.texts_to_sequences(np.array(listq1))
X_test_q1 = tokenizer.texts_to_sequences(X_tested['processed_text_q1'].values.astype(str))
X_test_q1 = pad_sequences(X_test_q1, maxlen = 30, padding='post')

# X_test_q2 = tokenizer.texts_to_sequences(np.array(listq2))
X_test_q2 = tokenizer.texts_to_sequences(X_tested['processed_text_q2'].values.astype(str))
X_test_q2 = pad_sequences(X_test_q2, maxlen = 30, padding='post')

In [ ]:
word_index = tokenizer.word_index

In [ ]:
embedding_index = {}
with open(path+'glove.6B.200d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [ ]:
embedding_matrix = np.random.random((len(word_index)+1, 200))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
model_q1 = tf.keras.Sequential()
model_q1.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q1.add(Bidirectional(LSTM(128, activation = 'tanh', return_sequences = True)))
model_q1.add(BatchNormalization())
model_q1.add(Dropout(0.01))
model_q1.add(Bidirectional(LSTM(128, return_sequences = True)))
model_q1.add(LSTM(128))
model_q1.add(BatchNormalization())
model_q1.add(Dense(60, activation = 'tanh'))
model_q1.add(Dense(2, activation = 'sigmoid'))

In [ ]:
model_q2 = tf.keras.Sequential()
model_q2.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q2.add(Bidirectional(LSTM(128, activation = 'tanh', return_sequences = True)))
model_q1.add(BatchNormalization())
model_q2.add(Dropout(0.01))
model_q2.add(Bidirectional(LSTM(128, return_sequences = True)))
model_q2.add(LSTM(128))
model_q1.add(BatchNormalization())
model_q2.add(Dense(60, activation = 'tanh'))
model_q2.add(Dense(2, activation = 'sigmoid'))

In [ ]:
mergedOut = Multiply()([model_q1.output, model_q2.output])

mergedOut = Flatten()(mergedOut)
mergedOut = Dense(100, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.01)(mergedOut)
mergedOut = Dense(50, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.01)(mergedOut)
mergedOut = Dense(2, activation = 'sigmoid')(mergedOut)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
new_model = Model([model_q1.input, model_q2.input], mergedOut)
new_model.compile(optimizer = Adam(learning_rate=0.001), loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True)
history = new_model.fit([X_train_q1, X_train_q2], y_trained,
                        validation_data=([X_test_q1, X_test_q2], y_tested),
                        epochs=10, batch_size=100, callbacks=[early_stopping])

Epoch 1/10
3235/3235 [==============================] - 151s 41ms/step - loss: 0.5390 - accuracy: 0.7287 - val_loss: 0.5344 - val_accuracy: 0.7320
Epoch 2/10
3235/3235 [==============================] - 104s 32ms/step - loss: 0.4782 - accuracy: 0.7669 - val_loss: 0.5319 - val_accuracy: 0.7308
Epoch 3/10
3235/3235 [==============================] - 104s 32ms/step - loss: 0.4314 - accuracy: 0.7908 - val_loss: 0.5036 - val_accuracy: 0.7611
Epoch 4/10
3235/3235 [==============================] - 102s 31ms/step - loss: 0.3862 - accuracy: 0.8157 - val_loss: 0.5743 - val_accuracy: 0.6748
Epoch 5/10
3235/3235 [==============================] - 102s 31ms/step - loss: 0.3397 - accuracy: 0.8401 - val_loss: 0.5785 - val_accuracy: 0.7507
Epoch 6/10
3235/3235 [==============================] - 101s 31ms/step - loss: 0.2955 - accuracy: 0.8646 - val_loss: 0.6020 - val_accuracy: 0.7444
Epoch 6: early stopping


# REAL WORLD TEST SET

In [ ]:
df_test= pd.read_csv(path+'real_test.csv')
df_test.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,1,1,1,Does higher education level mean higher income?,Does higher income level mean higher education...,0.0
1,2,1,1,How many Indians live in the GTA?,Are Indians the largest minority in the GTA?,0.0
2,3,1,1,Will AI replace humans?,Will humans be taken over by AI?,1.0
3,4,1,1,Was U.S. the first country to legalize the gun?,Was the gun first legalized by the United Stat...,1.0
4,5,2,2,Is Pacific Ocean the largest of the world ocea...,Is Pacific Ocean the deepest of the world ocea...,0.0


In [ ]:
df_test.dropna(inplace=True)

In [ ]:
df_test['processed_text_q1'] = process_text(df_test['question1'].astype(str))
df_test['processed_text_q2'] = process_text(df_test['question2'].astype(str))
df_test.head()

,id,qid1,qid2,question1,question2,is_duplicate,processed_text_q1,processed_text_q2
0,1,1,1,Does higher education level mean higher income?,Does higher income level mean higher education...,0.0,higher education level mean higher income,higher income level mean higher education level
1,2,1,1,How many Indians live in the GTA?,Are Indians the largest minority in the GTA?,0.0,many indian live gta,indian largest minority gta
2,3,1,1,Will AI replace humans?,Will humans be taken over by AI?,1.0,ai replace human,human taken ai
3,4,1,1,Was U.S. the first country to legalize the gun?,Was the gun first legalized by the United Stat...,1.0,u first country legalize gun,gun first legalized united state america
4,5,2,2,Is Pacific Ocean the largest of the world ocea...,Is Pacific Ocean the deepest of the world ocea...,0.0,pacific ocean largest world ocean basin,pacific ocean deepest world ocean basin


In [ ]:
X_tested_q1 = tokenizer.texts_to_sequences(df_test['processed_text_q1'].values.astype(str))
X_tested_q1 = pad_sequences(X_tested_q1, maxlen=30, padding='post')

X_tested_q2 = tokenizer.texts_to_sequences(df_test['processed_text_q2'].values.astype(str))
X_tested_q2 = pad_sequences(X_tested_q2, maxlen=30, padding='post')

In [ ]:
predictions = new_model.predict([X_tested_q1, X_tested_q2])
predictions = np.argmax(predictions, axis=1)
predictions

2/2 [==============================] - 0s 12ms/step


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
actual_labels = df_test['is_duplicate'].astype(int).values
actual_labels

array([0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0])

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(actual_labels, predictions))
print("Accuracy:", accuracy_score(actual_labels, predictions))

              precision    recall  f1-score   support

           0       0.65      0.93      0.76        28
           1       0.50      0.12      0.20        16

    accuracy                           0.64        44
   macro avg       0.57      0.53      0.48        44
weighted avg       0.60      0.64      0.56        44

Accuracy: 0.6363636363636364
